## Amazon Bedrock Guardrails: 
Amazon Bedrock Guardrails provides robust content filtering for generative AI applications. This customizable framework helps enterprises implement safety protocols for their AI applications by setting up filters to prevent harmful, inaccurate, or sensitive content. Guardrails enables control over topics, content appropriateness, and PII handling through configurable policies. Organizations can define what information should be blocked, filtered, or anonymized based on specific requirements. Built with regulatory compliance in mind, Guardrails allows developers to establish boundaries that keep AI interactions safe while maintaining functionality and utility. It integrates seamlessly with Bedrock's foundation models, providing protection without sacrificing performance or user experience.

in this notebook: we will perform the following 

## Basic Setup
- **UUID Generation**: Creates a unique client request token to ensure idempotency (prevents accidental duplicate guardrail creation)
- **Name and Description**: Sets clear identifiers for the guardrail's purpose ("AdvancedRagWorkshopGuardrails" focused on SEC filing content)
- **Blocked Messages**: Customized messages displayed when input or output is blocked by guardrails

## Topic Policy Configuration
- **Financial Advice (DENY)**: Blocks any content that recommends investments or financial decisions
- **Legal Interpretation (DENY)**: Prevents the model from interpreting legal or regulatory requirements
- **Future Performance**: There's a commented-out policy that would block speculation beyond disclosed forward-looking statements

## Content Policy Configuration
Sets high-strength filters for both input and output on:
- **HATE**: Blocks hateful content
- **INSULTS**: Filters insulting language
- **SEXUAL**: Prevents sexual content
- **VIOLENCE**: Blocks violent content
- **MISCONDUCT**: Filters content promoting illegal or harmful activities
- **PROMPT_ATTACK**: Only filters input (HIGH) but not output (NONE) for prompt injection attempts

## Contextual Grounding Policy
Sets thresholds for ensuring responses remain grounded to source material:
- **GROUNDING**: 0.1 threshold requires responses to be based on the provided context/documents
- **RELEVANCE**: 0.1 threshold ensures responses are relevant to the user's query

## Word Policy Configuration
Blocks specific words deemed inappropriate for the application:

## Sensitive Information Policy
### PII Entities
Configures ANONYMIZE action (redacts/masks content) for a comprehensive list of personal identifiable information types:
- Personal identifiers (NAME, EMAIL, PHONE, ADDRESS, AGE)
- Financial data (CREDIT_DEBIT_CARD details, BANK ACCOUNT numbers)
- Government IDs (SOCIAL_SECURITY_NUMBER, DRIVER_ID, PASSPORT_NUMBER)
- Technical identifiers (IP_ADDRESS, MAC_ADDRESS)
- Authentication data (PASSWORD, PIN, USERNAME)
- And many other PII types

### Regex Patterns
Custom regex patterns to catch domain-specific sensitive information:
- **Medical Record Numbers**: Pattern matching "123-456-7890" format
- **Insurance Numbers**: Pattern matching two letters followed by 7 digits

## Tags
Metadata for organizing and tracking the guardrail:
- **Environment**: "Production"
- **Department**: "Finance"

## Response Handling
Captures and outputs the:
- **Guardrail ID**: Unique identifier for the created guardrail
- **Guardrail ARN**: Full Amazon Resource Name
- **Version**: Version number of the created guardrail

## 1. Import Required Libraries

In [ ]:
# Import functions from advanced_rag_utils
from advanced_rag_utils import (
    load_variables,
    setup_bedrock_guardrails_client,
    create_guardrail_if_not_exists,
    create_guardrail_version,
    save_variables
)

## 2. Load Configuration Variables

In [ ]:
# Load variables from the configuration file
variables = load_variables("../variables.json")
variables  # Display the loaded variables for confirmation

## 3. Initialize Bedrock Client and Create a Guardrail

In [ ]:
# Initialize the Bedrock client for guardrails
bedrock = setup_bedrock_guardrails_client(variables['regionName'])

# Create a guardrail or get its ID if it already exists
guardrail_id = create_guardrail_if_not_exists(
    bedrock_client=bedrock,
    guardrail_name="AdvancedRagWorkshopGuardrails"
)

## 4. Save Guardrail ID to Configuration File

In [ ]:
# Update the variables dictionary with the new Guardrail ID
variables["guardrail_id"] = guardrail_id

# Save updated variables to the configuration file
save_variables(variables, "../variables.json")

## 5. Create a Published Version of the Guardrail

In [ ]:
# Create a published version of the guardrail
guardrail_version = create_guardrail_version(
    guardrail_id=guardrail_id,
    description="Production version 1.0",
    bedrock_client=bedrock
)

## 6. Save Guardrail Version to Configuration File

In [ ]:
# Update the variables dictionary with the new Guardrail Version
variables["guardrail_version"] = guardrail_version

# Save updated variables to the configuration file
save_variables(variables, "../variables.json")

> **Note**: After creating the guardrail, we recommend exploring its settings in the Amazon Bedrock console. The console provides a visual interface to understand how the guardrail is configured and allows you to test it interactively.
> 
> **[➡️ View your Guardrails in the AWS Console](https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/guardrails)**
>
> In the console, you can:
> - See all configured policies (topic filters, content filters, sensitive information policies)
> - Test sample prompts against your guardrail
> - View guardrail versions and their differences
> - Monitor usage metrics and blocked content statistics
> - Make adjustments to policies as needed